In [10]:
import re
import json
from typing import Dict, Optional, List, Tuple
import pymorphy3

# Инициализируем морфологический анализатор
morph = pymorphy3.MorphAnalyzer()


def normalize_text(text: str) -> str:
    """
    Нормализует текст, приводя все слова к начальной форме.
    
    Args:
        text: Исходный текст
        
    Returns:
        Нормализованный текст с словами в начальной форме
    """
    words = text.split()
    normalized_words = []
    
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word)
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            normalized_words.append(parsed.normal_form)
        else:
            normalized_words.append(word)
    
    return ' '.join(normalized_words)


def load_law_aliases_with_morphology():
    """
    Загружает law_aliases.json и создает индекс с нормализованными формами.
    ОПТИМИЗАЦИЯ: предкомпилирует все регулярные выражения один раз.
    
    Returns:
        Tuple[Dict, List]: (нормализованный_алиас -> [(оригинал, law_id)], все алиасы отсортированные)
    """
    with open('law_aliases.json', 'r', encoding='utf-8') as f:
        law_aliases = json.load(f)
    
    normalized_index = {}
    all_aliases = []
    
    for law_id, aliases in law_aliases.items():
        for alias in aliases:
            alias_lower = alias.lower()
            normalized = normalize_text(alias_lower)
            
            if normalized not in normalized_index:
                normalized_index[normalized] = []
            normalized_index[normalized].append((alias_lower, law_id))
            
            # ОПТИМИЗАЦИЯ: предкомпилируем регулярное выражение один раз
            # Используем границы слов только для буквенно-цифровых символов
            escaped_alias = re.escape(alias_lower)
            
            # Добавляем границу слова в начале, только если алиас начинается с буквы/цифры
            if re.match(r'[\w]', alias_lower):
                pattern = r'\b' + escaped_alias
            else:
                pattern = escaped_alias
            
            # Добавляем границу слова в конце, только если алиас заканчивается буквой/цифрой
            if re.search(r'[\w]$', alias_lower):
                pattern = pattern + r'\b'
            
            compiled_pattern = re.compile(pattern)
            
            all_aliases.append({
                'original': alias_lower,
                'normalized': normalized,
                'law_id': law_id,
                'length': len(alias_lower),
                'word_count': len(alias_lower.split()),
                'compiled_pattern': compiled_pattern  # Сохраняем скомпилированный паттерн
            })
    
    all_aliases.sort(key=lambda x: x['length'], reverse=True)
    
    return normalized_index, all_aliases


def find_law_in_text(text: str, normalized_index: Dict, all_aliases: List[Dict]) -> Optional[str]:
    """
    Ищет упоминание кодекса в тексте с учетом склонений.
    
    Args:
        text: Текст для поиска
        normalized_index: Индекс нормализованных форм
        all_aliases: Список всех алиасов
        
    Returns:
        law_id или None
    """
    text_lower = text.lower()
    
    # Сначала пробуем прямое совпадение (быстрее и точнее)
    # ОПТИМИЗАЦИЯ: используем предкомпилированные регулярные выражения
    for alias_data in all_aliases:
        # Используем уже скомпилированный паттерн - в разы быстрее!
        if alias_data['compiled_pattern'].search(text_lower):
            return alias_data['law_id']
    
    # Если прямого совпадения нет, пробуем с нормализацией
    # Разбиваем текст на фразы (последовательности слов)
    word_sequences = re.finditer(r'[а-яёА-ЯЁ\w\s]+', text_lower)
    
    for seq_match in word_sequences:
        sequence = seq_match.group()
        words = sequence.split()
        
        max_window = min(10, len(words))
        
        for window_size in range(max_window, 0, -1):
            for i in range(len(words) - window_size + 1):
                window = words[i:i + window_size]
                phrase = ' '.join(window)
                
                normalized_phrase = normalize_text(phrase)
                
                if normalized_phrase in normalized_index:
                    # Дополнительная проверка для коротких алиасов
                    matches = normalized_index[normalized_phrase]
                    
                    for original_alias, law_id in matches:
                        # Если алиас короткий (как "НК", "ГК"), требуем точного совпадения
                        if len(original_alias) <= 3:
                            # Проверяем, что фраза в тексте выглядит как аббревиатура
                            if re.search(r'\b' + re.escape(phrase.upper()) + r'\b', text.upper()):
                                return law_id
                        else:
                            return law_id
    
    return None


# Загружаем данные один раз
NORMALIZED_INDEX, ALL_ALIASES = load_law_aliases_with_morphology()


def parse_legal_reference_v2(text: str) -> Dict[str, Optional[str]]:
    """
    Парсит юридический текст и извлекает номера статьи, пункта и подпункта.
    Версия с поддержкой склонений через pymorphy3.
    
    Args:
        text: Текст с упоминанием статьи, пункта и/или подпункта
        
    Returns:
        Словарь с полями law_id, article, point_article, subpoint_article
    """
    result = {
        "law_id": None,
        "article": None,
        "point_article": None,
        "subpoint_article": None
    }
    
    text = text.lower().strip()
    
    # Поиск law_id с учетом склонений
    result["law_id"] = find_law_in_text(text, NORMALIZED_INDEX, ALL_ALIASES)
    
    # Паттерны для поиска статьи
    # Поддержка форматов: 115, 115.3, 17-2.1 и т.д. (обязательно заканчивается цифрой)
    article_patterns = [
        r'стать[яиеюй]?\s+(\d+(?:[.-]\d+)*)',
        r'ст\.?\s*(\d+(?:[.-]\d+)*)',
        r'(?<!пункт\s)(?<!пункта\s)(?<!подпункт\s)(?<!подпункта\s)(\d+(?:[.-]\d+)*)\s+стать[яиеюй]?',
        r'(?<!пункт\s)(?<!пункта\s)(?<!подпункт\s)(?<!подпункта\s)(\d+(?:[.-]\d+)*)\s+ст\.?(?!\s*\d)',
    ]
    
    # Паттерны для поиска пункта
    point_patterns = [
        r'(?<!под)пункт[аеуыои]?\s+(\d+[а-я]?)',
        r'(?<![а-я])п\.?\s*(\d+[а-я]?)',
    ]
    
    # Паттерны для поиска подпункта
    subpoint_patterns = [
        r'подпункт[аеуыои]?\s+([а-я\d]+)',
        r'подп\.?\s*([а-я\d]+)',
        r'пп\.?\s*([а-я\d]+)',
    ]
    
    # Поиск статьи
    for pattern in article_patterns:
        match = re.search(pattern, text)
        if match:
            result["article"] = match.group(1)
            break
    
    # Поиск пункта
    for pattern in point_patterns:
        match = re.search(pattern, text)
        if match:
            result["point_article"] = match.group(1)
            break
    
    # Поиск подпункта
    for pattern in subpoint_patterns:
        match = re.search(pattern, text)
        if match:
            result["subpoint_article"] = match.group(1)
            break
    
    return result


# Тестирование функции с расширенным набором тестов
test_cases_v2 = [
    "В ходе судебного заседания по вопросу наследования было установлено, что права истца на получение имущества защищены законодательством. В частности, это регулируется пп. ж п. 2 ст. 1506 Семейного кодекса РФ, что позволяет оспорить права других наследников. Суд принял решение в пользу истца, основываясь на данной норме законодательства.",
]

print("=" * 80)
print("ТЕСТИРОВАНИЕ ВЕРСИИ 2 (с поддержкой склонений через pymorphy3)")
print("=" * 80)
print()

for test in test_cases_v2:
    result = parse_legal_reference_v2(test)
    print(f"Входной текст: '{test}'")
    print(f"Результат: {result}")
    print()



ТЕСТИРОВАНИЕ ВЕРСИИ 2 (с поддержкой склонений через pymorphy3)

Входной текст: 'В ходе судебного заседания по вопросу наследования было установлено, что права истца на получение имущества защищены законодательством. В частности, это регулируется пп. ж п. 2 ст. 1506 Семейного кодекса РФ, что позволяет оспорить права других наследников. Суд принял решение в пользу истца, основываясь на данной норме законодательства.'
Результат: {'law_id': '8', 'article': '1506', 'point_article': '2', 'subpoint_article': 'ж'}

